In [1]:
subcases_pattern="1461_Nustar"
reference_location="../.."
source_name="Crab"
osa_version='OSA10.2'
nscw=3
mosaic=0
chi2_limit=1.2
systematic_fraction=0.01
search_time=15 #search time of INTEGRAL DATA around the time of observation (days)

In [2]:
if reference_location == 'None':
    import subprocess
    logfile_name = 'log_download_reference.txt'
    logfile = open(logfile_name, 'w')
    cmd = 'wget https://www.isdc.unige.ch/~ferrigno/Downloads/subcases_herX1_nustar.tgz;tar xfz subcases_herX1_nustar.tgz;rm subcases_herX1_nustar.tgz'
    # cmd='pwd;ls'
    out = subprocess.call(cmd, stdout=logfile, stderr=logfile, shell=True)
    logfile.close()
    if out != 0:
        print('Exit status of download and decompression is %d')
        raise RuntimeError
    
    reference_location='.'

#  ISGRI verification with NuSTAR reference

simultaneous observations allow easy comparisong

normalization need to be fitted and reported

In [3]:
%matplotlib notebook
#import matplotlib.pylab as plt


import numpy as np
import importlib
from astroquery.simbad import Simbad
from astropy import coordinates as coord
import os
import numpy as np
import glob

from astropy.io import fits as pf
from astropy.time import Time
from astropy.time import TimeDelta

from astropy import table



In [4]:

import requests
url="https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw/timesystem/api/v1.0/scwlist/cons/"
def get_scw_list(ra,dec,time, delta_time): #time in UTC T (2019-06-23T16:53:39.000), deltat in days
    
#     result_table = Simbad.query_object(source_name)
#     source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))

#     ra=source_coord.ra.deg
#     dec=source_coord.dec.deg
    
    dt = TimeDelta(delta_time)
    
    Tstart = Time(time) - dt
    Tstop  = Time(time) + dt
    
    Tstart.format='isot'
    Tstop.format='isot'
    
    
    params=Tstart.value+'/'+Tstop.value+'?&ra='+str(ra)+'&dec='+str(dec)+'&radius='+str(6.0)+'&min_good_isgri=1000'
    print(url+params)
    return requests.get(url+params).json() #,cookies=cookies).json()



In [5]:
reference_location

# if reference_location  not git or http may be a problem
# dependency injection for this, if can

'../..'

In [6]:

subcase=None
reference_spectra = None

for subcase_dir in glob.glob(reference_location+"/subcases/*"+subcases_pattern+"*"):
    print("inspecting", subcase_dir)
    
    fns = [os.path.basename(fn) for fn in sorted(glob.glob(subcase_dir+"/*"))]
    
    try:
        #If it is rebinned with my script
        reference_spectra = [fn for fn in fns if fn.endswith("rbn.pi")]
        
        reference_rmfs = [fn for fn in fns if fn.endswith("sr.rmf") or fn.endswith("sr.rmf.gz")]
        
        reference_arfs = [fn for fn in fns if fn.endswith("sr.arf")]
        
        reference_backs = [fn for fn in fns if fn.endswith("bk.pha")]
        
    except: 
        print("Not found Nustar Files in " + subcase_dir)
        continue
        
    
    try:
        print(subcase_dir+'/'+reference_spectra[0])
        spec=pf.open(subcase_dir+'/'+reference_spectra[0])

        t1=Time(spec[1].header['DATE-OBS'])
        t2=Time(spec[1].header['DATE-END'])

        ra=spec[1].header['RA_OBJ']
        dec=spec[1].header['DEC_OBJ']

        delta_time = (t2 - t1 )/2

        time=t1+delta_time

        spec.close()


        scwlist= get_scw_list(ra,dec, time, search_time)
        print(time)
        
        #print(scwlist)
    except:
        print("Not getting times")
        continue
    
   
        
    #print(fns)
    
    reference_dir=subcase_dir
    
    #print("found",reference_dir, reference_spectra, len(scwlist),len(ref_sources), ref_sources)
        
    break
        
if len(list(zip(reference_spectra, reference_backs, reference_rmfs, reference_arfs))) == 0:
    raise Exception(f"no compatible NuSTAR spectra found; last fns tried {fns}")
#reference_spectra, reference_rmfs


inspecting ../../subcases/Crab_1461_Nustar
../../subcases/Crab_1461_Nustar/FPMA_sr_rbn.pi
https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw/timesystem/api/v1.0/scwlist/cons/2014-09-17T06:26:07.000/2014-10-17T06:26:07.000?&ra=83.63303&dec=22.01447&radius=6.0&min_good_isgri=1000
2014-10-02T06:26:07.000


In [7]:
print("searching for coordinates of", source_name)

result_table = Simbad.query_object(source_name)
source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))

result_table

searching for coordinates of Crab


MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object
M 1,05 34 31.94,+22 00 52.2,6,6,--,--,0,C,R,2011A&A...533A..10L


In [8]:
import random
if nscw >0:
    random.seed(0)
    scw_pick = random.sample([
                s+"."+"001"
                for s in list(sorted(set( scwlist  ))) 
                if s.endswith("0010")
            ],nscw)


    scw_list_str = ",".join(scw_pick)
else:
    scw_list_str=",".join([s+"."+"001" for s in sorted(set( scwlist  ))])

#print(len(scw_list_str))

scw_list_str

'146200380010.001,146200470010.001,146100180010.001'

In [9]:
import oda_api.api
import importlib
importlib.reload(oda_api)

try:
    disp = oda_api.api.DispatcherAPI(host="http://cdcihn/staging-1.2/dispatcher")
    disp.get_instrument_description("isgri")
except:
    try:
        disp = oda_api.api.DispatcherAPI(host='https://www.astro.unige.ch/cdci/astrooda/dispatch-data')
        disp.get_instrument_description("isgri")
    except:
        raise ConnectionError






--------------
instrumet: isgri

--------------
query_name: src_query
 name: src_name,  value: test,  units: str, 
 name: RA,  value: 0.0,  units: deg, 
 name: DEC,  value: 0.0,  units: deg, 
 name: T1,  value: 2001-12-11T00:00:00.000,  units: None, 
 name: T2,  value: 2001-12-11T00:00:00.000,  units: None, 

--------------
query_name: isgri_parameters
 name: user_catalog,  value: None,  units: str, 
 name: scw_list,  value: [],  units: names_list, 
 name: selected_catalog,  value: None,  units: str, 
 name: radius,  value: 5.0,  units: deg, 
 name: max_pointings,  value: 50,  units: None, 
 name: osa_version,  value: None,  units: str, 
 name: E1_keV,  value: 10.0,  units: keV, 
 name: E2_keV,  value: 40.0,  units: keV, 

--------------
query_name: isgri_image_query
 product_name: isgri_image
 name: detection_threshold,  value: 0.0,  units: sigma, 
 name: image_scale_min,  value: None,  units: None, 
 name: image_scale_max,  value: None,  units: None, 

--------------
query_name: isg

In [10]:

mosaic=1

if mosaic == 1:

    for c_emin in [28]:

        image = disp.get_product(instrument="isgri", 
                         product="isgri_image", 
                         product_type="Real", 
                         osa_version=osa_version,
                         E1_keV=np.round(c_emin),
                         E2_keV=80.0,
                         scw_list=scw_list_str)

        sources=image.dispatcher_catalog_1.table[image.dispatcher_catalog_1.table['significance']>=6.0]
        #source = sources[sources['src_names']==source_name]
        unique_sources=table.unique(sources, keys=['src_names'])
        
        #print(source)

#         d=image.mosaic_image_0.data_unit[1].data


#         img=np.array(d.data)

#         m_bkg=img<10
#         m_bkg&=img!=0

#         img_std = np.std(img[m_bkg])

#         img[np.array(img)>img_std*5]=img_std*5

        #plt.imshow(img)

#         by_lt['%.10lg'%c_emin]=dict(
#             emin=c_emin,            
#             imgstd=img_std,
#         )
#         by_lt['%.10lg'%c_emin].update(dict([(n, source[n]) for n in source.colnames]))

        
#     for ltb, c in by_lt.items():
#         print(lt, c['significance'],c['significance']/c['imgstd'])

146200380010.001,146200470010.001,146100180010.001
- waiting for remote response, please wait run_analysis http://cdcihn/staging-1.2/dispatcher
osa_version OSA10.2
E1_keV 28
E2_keV 80.0
scw_list 146200380010.001,146200470010.001,146100180010.001
instrument isgri
product_type isgri_image
query_type Real
off_line (False,)
query_status ('new',)
verbose (False,)
session_id Q65ZT4WN6ISIGQ8J
dry_run (False,)
api True
oda_api_version 1.0.2


query done succesfully!


In [11]:
##Removes new sources and adds our if not found
FLAG=0
torm=[]
for ID,n in enumerate(unique_sources['src_names']):
    if(n[0:3]=='NEW'):
        torm.append(ID)
    if(n==source_name):
        FLAG=1

unique_sources.remove_rows(torm)
nrows=len(unique_sources['src_names'])

if FLAG==0:
    unique_sources.add_row((0,source_name,0,ra,dec,0,2,0,0))

image.dispatcher_catalog_1.table = unique_sources

api_cat=image.dispatcher_catalog_1.get_api_dictionary()



In [12]:
spectrum = disp.get_product(instrument="isgri", 
                 product="isgri_spectrum", 
                 product_type="Real", 
                 osa_version=osa_version,
                 E1_keV=25.0,
                 E2_keV=80.0,
                 scw_list=scw_list_str,
                 selected_cat=api_cat)

146200380010.001,146200470010.001,146100180010.001
- waiting for remote response, please wait run_analysis http://cdcihn/staging-1.2/dispatcher
osa_version OSA10.2
E1_keV 25.0
E2_keV 80.0
scw_list 146200380010.001,146200470010.001,146100180010.001
selected_cat {"cat_frame": "fk5", "cat_coord_units": "deg", "cat_column_list": [[3], ["Crab"], [345.2102355957031], [83.6318130493164], [22.014904022216797], [-32768], [2], [0], [0.0002800000074785203]], "cat_column_names": ["meta_ID", "src_names", "significance", "ra", "dec", "NEW_SOURCE", "ISGRI_FLAG", "FLAG", "ERR_RAD"], "cat_column_descr": [["meta_ID", "<i8"], ["src_names", "<U18"], ["significance", "<f8"], ["ra", "<f8"], ["dec", "<f8"], ["NEW_SOURCE", "<i8"], ["ISGRI_FLAG", "<i8"], ["FLAG", "<i8"], ["ERR_RAD", "|O"]], "cat_lat_name": "dec", "cat_lon_name": "ra"}
instrument isgri
product_type isgri_spectrum
query_type Real
off_line (False,)
query_status ('new',)
verbose (False,)
session_id TGEV49GW1UN9427Q
dry_run (False,)
api True
oda_ap

/pyenv/versions/3.8.2/lib/python3.8/site-packages/oda_api/api.py:326: UserWarning: 
----------------------------------------------------------------------------
the parameter: selected_cat   is not among valid ones:
['src_name', 'RA', 'DEC', 'T1', 'T2', 'scw_list', 'selected_catalog', 'radius', 'max_pointings', 'osa_version', 'E1_keV', 'E2_keV']
this will throw an error in a future version 
and might breack the current request!
 ----------------------------------------------------------------------------

  warnings.warn(msg)




query done succesfully!


In [13]:
specprod=[l for l in spectrum._p_list if l.meta_data['src_name'] == source_name]    

spec_fn="/tmp/isgri_spectrum_{}.fits".format(source_name.replace(' ', '_'))
arf_fn="/tmp/isgri_arf_{}.fits".format(source_name.replace(' ', '_'))
rmf_fn="/tmp/isgri_rmf_{}.fits".format(source_name.replace(' ', '_'))

specprod[0].write_fits_file(spec_fn)
specprod[1].write_fits_file(arf_fn)
specprod[2].write_fits_file(rmf_fn)

In [14]:
from astropy.io import fits

for s, b, r, a in zip(reference_spectra, reference_backs, reference_rmfs, reference_arfs):

    fs=fits.open(reference_dir+"/"+s)
    fr=fits.open(reference_dir+"/"+r)
    fb=fits.open(reference_dir+"/"+b)
    fa=fits.open(reference_dir+"/"+a)
    

    fs[2].header['RESPFILE'] = 'NONE'
    fs[2].header['ANCRFILE'] = 'NONE'
    fs[2].header['BACKFILE'] = 'NONE'
    
    #f[2].header['RESPFILE']
    if "A" in s:
        fn = "reference_spec_A.fits"
    else:
        fn = "reference_spec_B.fits"
        
    print("writing", fn)        
    fs.writeto(fn, overwrite=True)
    
    if 'A' in r:
        fn = "reference_rmf_A.fits"
    else:
        fn = "reference_rmf_B.fits"
        
    print("writing", fn)        
    fr.writeto(fn, overwrite=True)
    
    if 'A' in b:
        fn = "reference_bkg_A.fits"
    else:
        fn = "reference_bkg_B.fits"

    print("writing", fn)        
    fb.writeto(fn, overwrite=True)
    
    if 'A' in a:
        fn = "reference_arf_A.fits"
    else:
        fn = "reference_arf_B.fits"
        
    print("writing")
    fa.writeto(fn, overwrite=True)
    
    fs.close()
    fb.close()
    fa.close()
    fr.close()


writing reference_spec_A.fits
writing reference_rmf_A.fits
writing reference_bkg_A.fits
writing
writing reference_spec_B.fits
writing reference_rmf_B.fits
writing reference_bkg_B.fits
writing


In [15]:
reference_spec_A="reference_spec_A.fits"
reference_spec_B="reference_spec_B.fits"
reference_arf_A="reference_arf_A.fits"
reference_arf_B="reference_arf_B.fits"
reference_rmf_A="reference_rmf_A.fits"
reference_rmf_B="reference_rmf_B.fits"
reference_bkg_A="reference_bkg_A.fits"
reference_bkg_B="reference_bkg_B.fits"
isgri_spec=spec_fn
isgri_arf=arf_fn
isgri_rmf=rmf_fn